In [2]:
%pwd

'c:\\Users\\DELL\\Documents\\Projects\\research'

In [45]:
import os
os.chdir("../")

In [83]:
%pwd

'c:\\Users\\DELL\\Documents'

In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
def load_pdf_file(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

In [21]:
extracted_data = load_pdf_file('C:/Users/DELL/Documents/Projects/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf')

In [26]:
#extracted_data

In [22]:
#Split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20 )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [23]:
text_chunks = text_split(extracted_data)
print("length of text chunks", len(text_chunks))

length of text chunks 6973


In [24]:
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [5]:
embeddings = download_hugging_face_embeddings()

In [25]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [7]:
query_result


[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

for root, dirs, files in os.walk("C:/Users/DELL"):
    if ".env" in files:
        print("✅ Found .env at:", os.path.join(root, ".env"))
        break


✅ Found .env at: C:/Users/DELL\Documents\Projects\.env


In [27]:
from dotenv import load_dotenv
import os

# Explicitly load the .env file from the current directory
load_dotenv(dotenv_path=".env")

# Now try getting the key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()
print("🔑 Key:", OPENAI_API_KEY[:10] + "..." if OPENAI_API_KEY else "❌ MISSING")


🔑 Key: sk-proj-OI...


In [28]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [9]:
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
#OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
        metric="cosine"
    )


In [10]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [29]:

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone Index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

#from langchain.embeddings import HuggingFaceEmbeddings

#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [29]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone Index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs={"k":3})

In [32]:
retrieved_docs = retriever.invoke("what is Acne?")

In [33]:
retrieved_docs

[Document(id='b6f629bd-a568-4230-be67-71c546c92fbe', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'C:/Users/DELL/Documents/Projects/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='af3cb555-512d-4309-badb-9bacca541289', metadata={'author': '', 'creationdate': '2017-05-0

In [34]:
from langchain_openai import OpenAI
import os

llm = OpenAI(
    temperature=0.4,
    max_tokens=500,
    openai_api_key=os.getenv("OPENAI_API_KEY").strip()
)


In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't knw. Udse three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
       ("system", system_prompt),
       ("human", "{input}"), 
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}